In [1]:
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
import h5py as h5
import time
import sys
import copy
#Quick fudge to make import from ../Scripts work
import sys
sys.path.append('/Users/floorbroekgaarden/Projects/BHNS_project/Scripts')

from time import sleep
from IPython.display import clear_output, display
import pandas as pd 
import string
# # sys.path.append(pathPostProcessing+'/2_CosmicIntegration')

In [2]:
import ClassCosmicIntegrator  as CI #Given settings and redshifts returns rates (2D arrays) Loads the data
# import ClassEvents            as CE
# import ClassBayes             as CB
import coencodeVarious        as CV
from PostProcessingScripts import * 


In [3]:

def writeToCompactFile(DCOtype='BHBH', BPSmodelName='A', Optimistic='False', minz=0, maxz=1, resz=100, pathCOMPASOutput= '/Volumes/Virgo/DATA/BHNS/alpha0_1/'):
    """  
    Options:
    
    DCOtype = ['BHBH', 'BHNS', 'NSNS']
    BPSmodelName = alphabetic letter given to this binary population model variation
    
    """
  
    
    ############ CONSTANTS   ##########
    #MSSFR names
    
    GSMFs = ['Panter et al. (2004) Single', 'Furlong et al. (2015) Single', 'Furlong et al. (2015) Double']
    MZs   = [ 'Langer et al. (2006)'      , 'Langer et al. +offset (2006)', 'Ma et al. (2015)']
    SFRs  = ['Madau et al. (2014)'         ,'Strolger et al. (2004)',     'Madau et al. (2017)']

    MSSFRnameslist = []
    MSSFRnameslist.append('000') # add phenomenological 

    for ind_GSMF, GSMF in enumerate(GSMFs):
        ind_y = ind_GSMF + 1
        for ind_MZ, MZ in enumerate(MZs):
            ind_z = ind_MZ +1
            for ind_SFR, SFR in enumerate(SFRs):
                ind_x = ind_SFR+1
                MSSFRnameslist.append('%s%s%s'%(ind_x, ind_y, ind_z))
    
    # set naming right
    if DCOtype== 'BHBH':
        DCOtypeName='BBH'
    elif DCOtype =='NSNS':
        DCOtypeName ='BNS'
    elif DCOtype=='BHNS':
        DCOtypeName ='BHNS'
    
    Data = CI.CosmicIntegrator(COMPASpath = pathCOMPASOutput, DCOtypes=DCOtypeName,\
           minRedshift=minz,   maxRedshift=maxz, nrRedshiftBins=resz, optimistic=Optimistic, Cosmology='WMAP')



    # variables and header names for variables
    xvar = [Data.COMPAS.mass1, \
            Data.COMPAS.mass2, \
            Data.COMPAS.tc, \
            Data.COMPAS.metallicitySystems\
           ]
    xvarnames = ['mass1',\
                 'mass2',\
                 'tc',\
                 'metallicitySystems'\
                ]
    xvarunits = ['Msun',\
                 'Msun',\
                 'Myr',\
                 '#'\
                ]
   

    # nr of rows and columns that will be used:
    ncol_var = len(xvar)   
    ncol_MSSFR = len(MSSFRnameslist)
    ncol_Rate_det = 1
    ncol_Rate_z0 = 1

    nrows = len(Data.COMPAS.mass1) + 1 # +1 for units (see below)
    # store variables, and Observed and intrinsic rates for all MSSFR variations:
    ncol = ncol_var + ( ncol_MSSFR * (ncol_Rate_det+ncol_Rate_z0)) 
    df_placeholder = np.zeros((nrows, ncol)) # will be filled in loop: 
    
    headernames=[]
    units=[]
    for ind_s, ss in enumerate(xvarnames):
        headernames.append(ss)
        units.append(xvarunits[ind_s])

    for ind_s, ss in enumerate(MSSFRnameslist):
        headernames.append(ss+'_R_det')
        headernames.append(ss+'_R_z0')
        # add units for units header
        units.append(' yr{-1}')
        units.append('Gpc^{-3} yr{-1}')
        

        
    df = pd.DataFrame(data=df_placeholder, columns=headernames)   
    # add units at first row (index=0)
    df.iloc[0]=units
    
    
    #Calculate for Neijssel phenomenological model: ($\mu$000)
    # this has to go seperate since it is so different from the others 
    Data.MSSFR.Zprescription         = 'logNormal' 
    Data.MSSFR.SFRprescription       = 'Neijssel et al. (2019)'
    Data.MSSFR.logNormalPrescription = 'Neijssel Phenomenological'
    Data.MSSFR.GSMFprescription      = None
    Data.MSSFR.ZMprescription        = None
    Data.cosmologicalIntegration()

    # rate per system weighted by detection probability of LVK GW network
    Rate_det        =  np.sum(Data.PerSystemPerRedshift_ratesObserved*Data.COMPAS.weight, axis=0) # //floor weight
    # intrinsic rate at redshift 0 
    Rate_z0 = Data.PerSystemPerRedshift_ratesIntrinsic[0,:] * Data.COMPAS.weight
    
    # we only have to print the xvar once
    for ind_xvar, xvarname in enumerate(xvarnames):
        df[xvarname][1:]=xvar[ind_xvar]
        
    # print rates for phenomenological model to dataframe:
    mssfr_ind =0 
    mssfr = MSSFRnameslist[mssfr_ind]
    df[mssfr+'_R_det'][1:] = Rate_det
    df[mssfr+'_R_z0'][1:]  = Rate_z0
    
    
    # and now for the other MSSFR models: 

    for ind_GSMF, GSMF in enumerate(GSMFs):
             for ind_MZ, MZ in enumerate(MZs):
                 for ind_SFR, SFR in enumerate(SFRs):
                    mssfr_ind +=1 # go to next MSSFR 
#                     clear_output(wait=True)
                    print('now starting the ' + str(mssfr_ind+1) + 'th MSSFR out of ' + str(len(MSSFRnameslist)))
                    Data.MSSFR.Zprescription         = 'MZ_GSMF'
                    Data.MSSFR.SFRprescription       = SFR
    #                 Data.MSSFR.logNormalPrescription = logNormal[nrL]
                    Data.MSSFR.GSMFprescription      = GSMF
                    Data.MSSFR.ZMprescription        = MZ
                    Data.cosmologicalIntegration()

                    # rate per system weighted by detection probability of LVK GW network
                    Rate_det        =  np.sum(Data.PerSystemPerRedshift_ratesObserved*Data.COMPAS.weight, axis=0) # //floor weight
                    # intrinsic rate at redshift 0 
                    Rate_z0 = Data.PerSystemPerRedshift_ratesIntrinsic[0,:] * Data.COMPAS.weight

                   
                    mssfr =  MSSFRnameslist[mssfr_ind]
                    df[mssfr+'_R_det'][1:] = Rate_det
                    df[mssfr+'_R_z0'][1:]  = Rate_z0

    print('N DCO = ', len(Rate_z0))
    dfCSVname = 'CompactData_model_' + BPSmodelName + '_' + DCOtype + '.csv' 
    df.to_csv(dfCSVname) 
    del df
    del Data 
    
    return
    
    

In [6]:
DCOTypeList = ['BHBH', 'BHNS', 'NSNS']

NumberBPSmodels=10
alphabet = list(string.ascii_uppercase)
BPSnameslist = alphabet[:NumberBPSmodels]

Virgo         = '/Volumes/Virgo/DATA/BHNS/'
AndromedaBHNS = '/Volumes/Andromeda/DATA/BHNS/'
AndromedaAllDCO  = '/Volumes/Andromeda/DATA/AllDCO/'

BPSpathList = [Virgo+'Fiducial/',\
              Virgo+'Fiducial/',\
#                AndromedaAllDCO+'fiducial/',\
#               AndromedaAllDCO+'fiducial/',\
              Virgo+'alpha0_1/',\
              Virgo+'alpha_10/',\
               AndromedaBHNS+'unstableCaseBB/',\
               AndromedaBHNS+'rapid/',\
               AndromedaAllDCO+'zeroBHkick/',\
               AndromedaBHNS+'massTransferEfficiencyFixed_0_25/',\
               AndromedaBHNS+'massTransferEfficiencyFixed_0_5/',\
               AndromedaBHNS+'massTransferEfficiencyFixed_0_75/'\
              ]



for ind_DCO, DCOtype in enumerate(DCOTypeList):
    print('----------------------------')
    print('now at DCO type ', DCOtype)
    print()
    if DCOtype=='BHBH':
        minz, maxz = 0,2
    elif DCOtype=='BHNS':
        minz, maxz = 0,2        
    elif DCOtype=='NSNS':
        minz, maxz = 0,2          
    
    for ind_BPS, BPSname in enumerate(BPSnameslist):
        print('now at BPS model name', BPSname)
        
        if BPSname=='B':
            Optimistic=True
            print('running optimistic CE for model ', BPSname )
        else:
            Optimistic=False
        # write to CSV file:
        writeToCompactFile(DCOtype=DCOtype,\
                           BPSmodelName=BPSname,\
                           Optimistic=Optimistic, \
                           minz=minz, maxz=maxz, resz=50,\
                           pathCOMPASOutput=BPSpathList[ind_BPS]\
                          )
        
    
  

----------------------------
now at DCO type  BHBH

now at BPS model name A
weighted samples :-D
Remember to  self.setCOMPASDCOmask() adag and self.setCOMPASData()
now starting the 2th MSSFR out of 28
now starting the 3th MSSFR out of 28
now starting the 4th MSSFR out of 28
now starting the 5th MSSFR out of 28
now starting the 6th MSSFR out of 28
now starting the 7th MSSFR out of 28
now starting the 8th MSSFR out of 28
now starting the 9th MSSFR out of 28
now starting the 10th MSSFR out of 28
now starting the 11th MSSFR out of 28
now starting the 12th MSSFR out of 28
now starting the 13th MSSFR out of 28
now starting the 14th MSSFR out of 28
now starting the 15th MSSFR out of 28
now starting the 16th MSSFR out of 28
now starting the 17th MSSFR out of 28
now starting the 18th MSSFR out of 28
now starting the 19th MSSFR out of 28
now starting the 20th MSSFR out of 28
now starting the 21th MSSFR out of 28
now starting the 22th MSSFR out of 28
now starting the 23th MSSFR out of 28
now star

now starting the 2th MSSFR out of 28
now starting the 3th MSSFR out of 28
now starting the 4th MSSFR out of 28
now starting the 5th MSSFR out of 28
now starting the 6th MSSFR out of 28
now starting the 7th MSSFR out of 28
now starting the 8th MSSFR out of 28
now starting the 9th MSSFR out of 28
now starting the 10th MSSFR out of 28
now starting the 11th MSSFR out of 28
now starting the 12th MSSFR out of 28
now starting the 13th MSSFR out of 28
now starting the 14th MSSFR out of 28
now starting the 15th MSSFR out of 28
now starting the 16th MSSFR out of 28
now starting the 17th MSSFR out of 28
now starting the 18th MSSFR out of 28
now starting the 19th MSSFR out of 28
now starting the 20th MSSFR out of 28
now starting the 21th MSSFR out of 28
now starting the 22th MSSFR out of 28
now starting the 23th MSSFR out of 28
now starting the 24th MSSFR out of 28
now starting the 25th MSSFR out of 28
now starting the 26th MSSFR out of 28
now starting the 27th MSSFR out of 28
now starting the 28t

now starting the 5th MSSFR out of 28
now starting the 6th MSSFR out of 28
now starting the 7th MSSFR out of 28
now starting the 8th MSSFR out of 28
now starting the 9th MSSFR out of 28
now starting the 10th MSSFR out of 28
now starting the 11th MSSFR out of 28
now starting the 12th MSSFR out of 28
now starting the 13th MSSFR out of 28
now starting the 14th MSSFR out of 28
now starting the 15th MSSFR out of 28
now starting the 16th MSSFR out of 28
now starting the 17th MSSFR out of 28
now starting the 18th MSSFR out of 28
now starting the 19th MSSFR out of 28
now starting the 20th MSSFR out of 28
now starting the 21th MSSFR out of 28
now starting the 22th MSSFR out of 28
now starting the 23th MSSFR out of 28
now starting the 24th MSSFR out of 28
now starting the 25th MSSFR out of 28
now starting the 26th MSSFR out of 28
now starting the 27th MSSFR out of 28
now starting the 28th MSSFR out of 28
N DCO =  685795
now at BPS model name F
weighted samples :-D
Remember to  self.setCOMPASDCOma

now starting the 8th MSSFR out of 28
now starting the 9th MSSFR out of 28
now starting the 10th MSSFR out of 28
now starting the 11th MSSFR out of 28
now starting the 12th MSSFR out of 28
now starting the 13th MSSFR out of 28
now starting the 14th MSSFR out of 28
now starting the 15th MSSFR out of 28
now starting the 16th MSSFR out of 28
now starting the 17th MSSFR out of 28
now starting the 18th MSSFR out of 28
now starting the 19th MSSFR out of 28
now starting the 20th MSSFR out of 28
now starting the 21th MSSFR out of 28
now starting the 22th MSSFR out of 28
now starting the 23th MSSFR out of 28
now starting the 24th MSSFR out of 28
now starting the 25th MSSFR out of 28
now starting the 26th MSSFR out of 28
now starting the 27th MSSFR out of 28
now starting the 28th MSSFR out of 28
N DCO =  45997
now at BPS model name C
weighted samples :-D
Remember to  self.setCOMPASDCOmask() adag and self.setCOMPASData()
now starting the 2th MSSFR out of 28
now starting the 3th MSSFR out of 28
now

now starting the 13th MSSFR out of 28
now starting the 14th MSSFR out of 28
now starting the 15th MSSFR out of 28
now starting the 16th MSSFR out of 28
now starting the 17th MSSFR out of 28
now starting the 18th MSSFR out of 28
now starting the 19th MSSFR out of 28
now starting the 20th MSSFR out of 28
now starting the 21th MSSFR out of 28
now starting the 22th MSSFR out of 28
now starting the 23th MSSFR out of 28
now starting the 24th MSSFR out of 28
now starting the 25th MSSFR out of 28
now starting the 26th MSSFR out of 28
now starting the 27th MSSFR out of 28
now starting the 28th MSSFR out of 28
N DCO =  14346
now at BPS model name J
weighted samples :-D
Remember to  self.setCOMPASDCOmask() adag and self.setCOMPASData()
now starting the 2th MSSFR out of 28
now starting the 3th MSSFR out of 28
now starting the 4th MSSFR out of 28
now starting the 5th MSSFR out of 28
now starting the 6th MSSFR out of 28
now starting the 7th MSSFR out of 28
now starting the 8th MSSFR out of 28
now st

In [ ]:
BPSmodelName ='C'
DCOtype='BHBH'

df = pd.read_csv('CompactData_model_' + BPSmodelName + '_' + DCOtype + '.csv', header=[0], skiprows=[1])

print(len(df['mass1']))
print((df['mass1']))

In [7]:
DCOTypeList = ['BHBH', 'BHNS', 'NSNS']

NumberBPSmodels=10
alphabet = list(string.ascii_uppercase)
BPSnameslist = alphabet[:NumberBPSmodels]

Virgo         = '/Volumes/Virgo/DATA/BHNS/'
AndromedaBHNS = '/Volumes/Andromeda/DATA/BHNS/'
AndromedaAllDCO  = '/Volumes/Andromeda/DATA/AllDCO/'

BPSpathList = [AndromedaAllDCO+'fiducial/',\
              AndromedaAllDCO+'fiducial/',\
#               Virgo+'alpha0_1/',\
#               Virgo+'alpha_10/',\
#                AndromedaBHNS+'unstableCaseBB/',\
#                AndromedaBHNS+'rapid/',\
#                AndromedaAllDCO+'zeroBHkick/',\
#                AndromedaBHNS+'massTransferEfficiencyFixed_0_25/',\
#                AndromedaBHNS+'massTransferEfficiencyFixed_0_5/',\
#                AndromedaBHNS+'massTransferEfficiencyFixed_0_75/'\
              ]



for ind_DCO, DCOtype in enumerate(DCOTypeList):
    print('----------------------------')
    print('now at DCO type ', DCOtype)
    print()
    if DCOtype=='BHBH':
        minz, maxz = 0,2
    elif DCOtype=='BHNS':
        minz, maxz = 0,2        
    elif DCOtype=='NSNS':
        minz, maxz = 0,2          
    
    for ind_BPS, BPSname in enumerate(BPSnameslist[0:2]):
        print('now at BPS model name', BPSname)
        
        if BPSname=='B':
            Optimistic=True
            print('running optimistic CE for model ', BPSname )
        else:
            Optimistic=False
        # write to CSV file:
        writeToCompactFile(DCOtype=DCOtype,\
                           BPSmodelName=BPSname,\
                           Optimistic=Optimistic, \
                           minz=minz, maxz=maxz, resz=50,\
                           pathCOMPASOutput=BPSpathList[ind_BPS]\
                          )
        

----------------------------
now at DCO type  BHBH

now at BPS model name A
weighted samples :-D
Remember to  self.setCOMPASDCOmask() adag and self.setCOMPASData()
now starting the 2th MSSFR out of 28
now starting the 3th MSSFR out of 28
now starting the 4th MSSFR out of 28
now starting the 5th MSSFR out of 28
now starting the 6th MSSFR out of 28
now starting the 7th MSSFR out of 28
now starting the 8th MSSFR out of 28
now starting the 9th MSSFR out of 28
now starting the 10th MSSFR out of 28
now starting the 11th MSSFR out of 28
now starting the 12th MSSFR out of 28
now starting the 13th MSSFR out of 28
now starting the 14th MSSFR out of 28
now starting the 15th MSSFR out of 28
now starting the 16th MSSFR out of 28
now starting the 17th MSSFR out of 28
now starting the 18th MSSFR out of 28
now starting the 19th MSSFR out of 28
now starting the 20th MSSFR out of 28
now starting the 21th MSSFR out of 28
now starting the 22th MSSFR out of 28
now starting the 23th MSSFR out of 28
now star

# MODEL K & L 

In [13]:
DCOTypeList = ['BHBH', 'BHNS', 'NSNS']

NumberBPSmodels=12
alphabet = list(string.ascii_uppercase)
BPSnameslist = alphabet[:NumberBPSmodels]

print(BPSnameslist[10:12])

Virgo         = '/Volumes/Virgo/DATA/BHNS/'
AndromedaBHNS = '/Volumes/Andromeda/DATA/BHNS/'
AndromedaAllDCO  = '/Volumes/Andromeda/DATA/AllDCO/'

BPSpathList = [\
#                 AndromedaAllDCO+'fiducial/',\
#               AndromedaAllDCO+'fiducial/',\
#               Virgo+'alpha0_1/',\
#               Virgo+'alpha_10/',\
#                AndromedaBHNS+'unstableCaseBB/',\
#                AndromedaBHNS+'rapid/',\
#                AndromedaAllDCO+'zeroBHkick/',\
#                AndromedaBHNS+'massTransferEfficiencyFixed_0_25/',\
#                AndromedaBHNS+'massTransferEfficiencyFixed_0_5/',\
#                AndromedaBHNS+'massTransferEfficiencyFixed_0_75/'\
                 AndromedaAllDCO + 'ccSNkick_30km_s/',\
                 AndromedaAllDCO + 'ccSNkick_100km_s/'
              ]



for ind_DCO, DCOtype in enumerate(DCOTypeList):
    print('----------------------------')

print('now at DCO type ', DCOtype)
    print()
    if DCOtype=='BHBH':
        minz, maxz = 0,2
    elif DCOtype=='BHNS':
        minz, maxz = 0,2        
    elif DCOtype=='NSNS':
        minz, maxz = 0,2          
    
    for ind_BPS, BPSname in enumerate(BPSnameslist[10:12]):
        print('now at BPS model name', BPSname)
        
        if BPSname=='B':
            Optimistic=True
            print('running optimistic CE for model ', BPSname )
        else:
            Optimistic=False
        # write to CSV file:
        writeToCompactFile(DCOtype=DCOtype,\
                           BPSmodelName=BPSname,\
                           Optimistic=Optimistic, \
                           minz=minz, maxz=maxz, resz=50,\
                           pathCOMPASOutput=BPSpathList[ind_BPS]\
                          )
        

['K', 'L']
----------------------------
now at DCO type  BHBH

now at BPS model name K
weighted samples :-D
Remember to  self.setCOMPASDCOmask() adag and self.setCOMPASData()
now starting the 2th MSSFR out of 28
now starting the 3th MSSFR out of 28
now starting the 4th MSSFR out of 28
now starting the 5th MSSFR out of 28
now starting the 6th MSSFR out of 28
now starting the 7th MSSFR out of 28
now starting the 8th MSSFR out of 28
now starting the 9th MSSFR out of 28
now starting the 10th MSSFR out of 28
now starting the 11th MSSFR out of 28
now starting the 12th MSSFR out of 28
now starting the 13th MSSFR out of 28
now starting the 14th MSSFR out of 28
now starting the 15th MSSFR out of 28
now starting the 16th MSSFR out of 28
now starting the 17th MSSFR out of 28
now starting the 18th MSSFR out of 28
now starting the 19th MSSFR out of 28
now starting the 20th MSSFR out of 28
now starting the 21th MSSFR out of 28
now starting the 22th MSSFR out of 28
now starting the 23th MSSFR out of 

IOError: Unable to create file (unable to open file: name = '/Volumes/Andromeda/DATA/AllDCO/ccSNkick_100km_s/COMPASOutput.h5', errno = 17, error message = 'File exists', flags = 15, o_flags = a02)

In [5]:
DCOTypeList = ['BHBH', 'BHNS', 'NSNS']

NumberBPSmodels=10
alphabet = list(string.ascii_uppercase)
BPSnameslist = alphabet[:NumberBPSmodels]

Virgo         = '/Volumes/Virgo/DATA/BHNS/'
AndromedaBHNS = '/Volumes/Andromeda/DATA/BHNS/'
AndromedaAllDCO  = '/Volumes/Andromeda/DATA/AllDCO/'

BPSpathList = [\
#                 AndromedaAllDCO+'fiducial/',\
#               AndromedaAllDCO+'fiducial/',\
#               Virgo+'alpha0_1/',\
#               Virgo+'alpha_10/',\
#                AndromedaBHNS+'unstableCaseBB/',\
#                AndromedaBHNS+'rapid/',\
#                AndromedaAllDCO+'zeroBHkick/',\
#                AndromedaBHNS+'massTransferEfficiencyFixed_0_25/',\
#                AndromedaBHNS+'massTransferEfficiencyFixed_0_5/',\
#                AndromedaBHNS+'massTransferEfficiencyFixed_0_75/'\
#                  AndromedaAllDCO + 'ccSNkick_30km_s/',\
                 AndromedaAllDCO + 'ccSNkick_100km_s/'
              ]





for ind_DCO, DCOtype in enumerate(['BHBH','BHNS', 'NSNS']):
    print('----------------------------')
    print('now at DCO type ', DCOtype)
    print()
    if DCOtype=='BHBH':
        minz, maxz = 0,2
    elif DCOtype=='BHNS':
        minz, maxz = 0,2        
    elif DCOtype=='NSNS':
        minz, maxz = 0,2          
    
    for ind_BPS, BPSname in enumerate(['K']):
        print('now at BPS model name', BPSname)
        
        if BPSname=='B':
            Optimistic=True
            print('running optimistic CE for model ', BPSname )
        else:
            Optimistic=False
        # write to CSV file:
        writeToCompactFile(DCOtype=DCOtype,\
                           BPSmodelName=BPSname,\
                           Optimistic=Optimistic, \
                           minz=minz, maxz=maxz, resz=50,\
                           pathCOMPASOutput=BPSpathList[ind_BPS]\
                          )
        

----------------------------
now at DCO type  BHBH

now at BPS model name K
weighted samples :-D
Remember to  self.setCOMPASDCOmask() adag and self.setCOMPASData()
now starting the 2th MSSFR out of 28
now starting the 3th MSSFR out of 28
now starting the 4th MSSFR out of 28
now starting the 5th MSSFR out of 28
now starting the 6th MSSFR out of 28
now starting the 7th MSSFR out of 28
now starting the 8th MSSFR out of 28
now starting the 9th MSSFR out of 28
now starting the 10th MSSFR out of 28
now starting the 11th MSSFR out of 28
now starting the 12th MSSFR out of 28
now starting the 13th MSSFR out of 28
now starting the 14th MSSFR out of 28
now starting the 15th MSSFR out of 28
now starting the 16th MSSFR out of 28
now starting the 17th MSSFR out of 28
now starting the 18th MSSFR out of 28
now starting the 19th MSSFR out of 28
now starting the 20th MSSFR out of 28
now starting the 21th MSSFR out of 28
now starting the 22th MSSFR out of 28
now starting the 23th MSSFR out of 28
now star

In [ ]:
Data = Data_modelC_BHNS

xvar = [Data.COMPAS.mass1, \
        Data.COMPAS.mass2, \
        Data.COMPAS.tc, \
        Data.COMPAS.metallicitySystems\
       ]

xvarnames = ['mass1',\
             'mass2',\
             'tc',\
             'metallicitySystems'\
            ]


ncol_var = len(xvar)
ncol_MSSFR = len(MSSFRnameslist)
ncol_Rate_det = 1
ncol_Rate_z0 = 1



nrows = len(Data_modelC_BHNS.COMPAS.mass1)
ncol = ncol_var + ( ncol_MSSFR * (ncol_Rate_det+ncol_Rate_z0)) 
df_placeholder = np.zeros((nrows, ncol))

print np.shape(df_placeholder)

In [ ]:

# headernames = 
headernames=[]
unitslist = []
xvarunits = ['Msun', 'Msun', 'Myr', '#']
for ind_s, ss in enumerate(xvarnames):
    headernames.append(ss)
    unitslist.append(xvarunits[ind_s])

for ind_s, ss in enumerate(MSSFRnameslist):
    headernames.append(ss+'_R_det')
    headernames.append(ss+'_R_z0')
    unitslist.append(' yr{-1}')
    unitslist.append('Gpc^{-3} yr{-1}')
df = pd.DataFrame(data=df_placeholder, columns=headernames)
print(headernames)
print(len(headernames))

In [ ]:
print(unitslist)

In [ ]:
# df = pd.DataFrame(data=df_placeholder, columns=headernames)
# # print(df)


df['mass1'][1:] = Data_modelC_BHNS.COMPAS.mass1[1:]
print(df)




In [ ]:
Data = Data_modelC_BHNS

xvar = [Data.COMPAS.mass1, \
        Data.COMPAS.mass2, \
        Data.COMPAS.tc, \
        Data.COMPAS.metallicitySystems\
       ]

ncol_var = len(xvar)
ncol_MSSFR = len(MSSFRnameslist)
ncol_Rate_obs = 1
ncol_Rate_z0 = 1

ncol = ncol_var + ( ncol_MSSFR * (ncol_Rate_obs*ncol_Rate_z0)) 



datafiller = np.zeros()

In [ ]:
df = pd.DataFrame(data=datas, index=NAMES).T